In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [5]:
# Defining the neural network model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1) 
        self.bn1 = nn.BatchNorm2d(num_features=8)
        self.tns1 = nn.Conv2d(in_channels=8, out_channels=4, kernel_size=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, padding=1) 
        self.bn2 = nn.BatchNorm2d(num_features=8)  
        self.pool1 = nn.MaxPool2d(2, 2)   
        self.conv3 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1) 
        self.bn3 = nn.BatchNorm2d(num_features=16) 
        self.conv4 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1) 
        self.bn4 = nn.BatchNorm2d(num_features=32)
        self.pool2 = nn.MaxPool2d(2, 2) 
        self.tns2 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=16, out_channels=64, kernel_size=3, padding=1) 
        self.bn5 = nn.BatchNorm2d(num_features=64) 
        self.conv6 = nn.Conv2d(in_channels=64, out_channels=10, kernel_size=1, padding=1)    
        self.gpool = nn.AvgPool2d(kernel_size=7)
        self.drop = nn.Dropout2d(0.1)

    def forward(self, x):
        x = self.tns1(self.drop(self.bn1(F.relu(self.conv1(x)))))
        x = self.drop(self.bn2(F.relu(self.conv2(x))))
        x = self.pool1(x)
        x = self.drop(self.bn3(F.relu(self.conv3(x))))        
        x = self.drop(self.bn4(F.relu(self.conv4(x))))
        x = self.tns2(self.pool2(x))
        x = self.drop(self.bn5(F.relu(self.conv5(x))))
        x = self.conv6(x)
        x = self.gpool(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)


In [6]:
# Setting the device as CUDA if available or else CPU and then generating the model summary for the number of parameters.
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
         Dropout2d-3            [-1, 8, 28, 28]               0
            Conv2d-4            [-1, 4, 30, 30]              36
            Conv2d-5            [-1, 8, 30, 30]             296
       BatchNorm2d-6            [-1, 8, 30, 30]              16
         Dropout2d-7            [-1, 8, 30, 30]               0
         MaxPool2d-8            [-1, 8, 15, 15]               0
            Conv2d-9           [-1, 16, 15, 15]           1,168
      BatchNorm2d-10           [-1, 16, 15, 15]              32
        Dropout2d-11           [-1, 16, 15, 15]               0
           Conv2d-12           [-1, 32, 15, 15]           4,640
      BatchNorm2d-13           [-1, 32, 15, 15]              64
        Dropout2d-14           [-1, 32,

<ipython-input-5-2299abe4ccf7>:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [7]:
torch.manual_seed(1)
# setting the batch size for each epoch
batch_size = 64

# Defining the train and test data loaders and the different transformations that need to be applied.
# Here we are converting the images to tensors and standarding the values between 0 and 1 based on the mean and standard deviation.
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [8]:
# Defining the train function which has the optimizer for gradients which is back propogated based on the negative log likelihood.
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

# Defining the test function to make predictions and calculate the accuracy based on the predictions and the targets
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:
# Sending the model to the device and setting the optimizer with default values
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# iterating through each epoch and passing the data to the train and test functions and printing the accuracy.
for epoch in range(0, 20):
    print("Epoch-", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Epoch- 0


  0%|          | 0/938 [00:00<?, ?it/s]<ipython-input-5-2299abe4ccf7>:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.10178203135728836 batch_id=937: 100%|██████████| 938/938 [00:29<00:00, 32.12it/s]



Test set: Average loss: 0.1005, Accuracy: 9728/10000 (97.3%)

Epoch- 1


loss=0.11127413064241409 batch_id=937: 100%|██████████| 938/938 [00:26<00:00, 35.55it/s]



Test set: Average loss: 0.0531, Accuracy: 9843/10000 (98.4%)

Epoch- 2


...


Test set: Average loss: 0.0192, Accuracy: 9933/10000 (99.3%)

Epoch- 18


loss=0.004224067088216543 batch_id=937: 100%|██████████| 938/938 [00:23<00:00, 39.17it/s]



Test set: Average loss: 0.0177, Accuracy: 9947/10000 (99.5%)

Epoch- 19


loss=0.036723729223012924 batch_id=937: 100%|██████████| 938/938 [00:24<00:00, 38.40it/s]



Test set: Average loss: 0.0184, Accuracy: 9938/10000 (99.4%)

